# Анализ тональности текста на NLTK

Это проект по анализу тональности. Он показывает несколько подходов: готовый лексикон VADER, классификацию субъективности и модель полярности на корпусе `movie_reviews`.

**Что внутри:**
- Быстрый baseline через VADER.
- Субъективность (objective vs subjective) на корпусе `subjectivity`.
- Полярность (positive vs negative) на корпусе `movie_reviews`.
- Мини-выводы и идеи для развития проекта.


## Установка и подготовка окружения

Сначала потребуется скачать датасеты NLTK.

```bash
pip install nltk
```


In [ ]:
import random
import nltk
from nltk.corpus import stopwords, subjectivity, movie_reviews
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.sentiment.util import extract_unigram_feats, mark_negation
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('subjectivity')
nltk.download('movie_reviews')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


## 1. Baseline: готовый лексикон VADER

VADER — быстрый метод, который оценивает тональность без обучения.


In [ ]:
texts = [
    "Обожаю этот курс, он очень полезный и понятный!",
    "Сервис работал ужасно, всё зависало и было медленно.",
    "Фильм оказался неплохим, но ожидал большего.",
]

sia = SentimentIntensityAnalyzer()

for text in texts:
    scores = sia.polarity_scores(text)
    print(text)
    print(scores)
    print('-' * 60)


## 2. Субъективность: обучаемый классификатор

Используем корпус `subjectivity`, где тексты помечены как `subj` (субъективные) и `obj` (объективные).


In [ ]:
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')]
documents = subj_docs + obj_docs
random.shuffle(documents)

all_words = nltk.FreqDist(word.lower() for word in subjectivity.words())
word_features = list(all_words)[:2000]

def subjectivity_features(doc):
    doc = mark_negation(doc)
    return extract_unigram_feats(doc, word_features)

featuresets = [(subjectivity_features(d), c) for (d, c) in documents]
train_set = featuresets[:1600]
test_set = featuresets[1600:]

subj_classifier = NaiveBayesClassifier.train(train_set)
print('Accuracy:', accuracy(subj_classifier, test_set))
subj_classifier.show_most_informative_features(10)


## 3. Полярность: классификация отзывов (movie_reviews)

Построим собственный классификатор для позитивных/негативных отзывов.


In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def to_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    if tag.startswith('V'):
        return wordnet.VERB
    if tag.startswith('N'):
        return wordnet.NOUN
    if tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN

def normalize(text):
    tokens = [t.lower() for t in word_tokenize(text) if t.isalpha()]
    tokens = [t for t in tokens if t not in stop_words]
    tagged = nltk.pos_tag(tokens)
    return [lemmatizer.lemmatize(word, to_wordnet_pos(tag)) for word, tag in tagged]

documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        words = normalize(movie_reviews.raw(fileid))
        documents.append((words, category))

random.shuffle(documents)

all_words = nltk.FreqDist(word for words, _ in documents for word in words)
word_features = list(all_words)[:2000]

def document_features(document_words):
    document_words = set(document_words)
    return {f'contains({word})': (word in document_words) for word in word_features}

featuresets = [(document_features(d), c) for (d, c) in documents]
train_set = featuresets[:1500]
test_set = featuresets[1500:]

polarity_classifier = NaiveBayesClassifier.train(train_set)
print('Accuracy:', accuracy(polarity_classifier, test_set))
polarity_classifier.show_most_informative_features(10)


## 4. Применение модели к новым текстам

Ниже — пример, как использовать обученный классификатор на новых отзывах.


In [ ]:
samples = [
    "This movie is a masterpiece with brilliant acting.",
    "The plot is boring and the pacing is terrible.",
]

for text in samples:
    words = normalize(text)
    prediction = polarity_classifier.classify(document_features(words))
    print(text)
    print('Prediction:', prediction)
    print('-' * 60)


## Итоги

- **VADER** отлично подходит для быстрых оценок без обучения.
- **Naive Bayes + `subjectivity`** позволяет определять субъективность.
- **Movie Reviews** — пример полноценной пайплайн-модели: очистка, лемматизация, обучение.

### Идеи для развития
- Добавить TF-IDF или word embeddings.
- Попробовать SVM/Logistic Regression.
- Подключить визуализацию метрик (confusion matrix).
